In [255]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF
%matplotlib inline

train_path = 'C://Users//lenovo//Desktop//Docs//ZS//dataset'
os.chdir(train_path)

train = pd.read_csv('yds_train2018.csv')
test = pd.read_csv('yds_test2018.csv')
promotional_expense = pd.read_csv('promotional_expense.csv')
holidays = pd.read_excel('holidays.xlsx', sheet_name = None)
sample = pd.read_csv('sample_submission.csv')

promotional = promotional_expense.groupby(['Month', 'Year', 'Country'], as_index = False)['Expense_Price'].sum()
train_by_month = train.groupby(['Year', 'Month', 'Product_ID', 'Country'], as_index = False)['Sales'].sum()

train_size = train_by_month.shape[0]
all_data = pd.concat([train_by_month, test.drop(['S_No'], axis = 1)], axis = 0)

def dateConverter(date):
    l = list(map(int, date.split(',')))
    return l[0], l[1], l[2]

holidays[['yr', 'mn', 'dt']] = holidays['Date'].apply(dateConverter).apply(pd.Series)
data_expense = pd.merge(all_data, promotional, how = 'left', on = ['Month', 'Year', 'Country'])
holidays['Date'], holidays['Month'], holidays['Year'] = holidays['dt'], holidays['mn'], holidays['yr']
holidays_count = holidays.groupby(['Year', 'Month', 'Country'], as_index = False)['Holiday'].count()

data_expense_hol = pd.merge(data_expense, holidays_count, how = 'left', on = ['Month', 'Year', 'Country'])

data_expense_hol['Holiday'].fillna(0, inplace = True)

data_expense_hol['Country'] = pd.get_dummies(data_expense_hol['Country'])
data_expense_hol['Month'] = pd.get_dummies(data_expense_hol['Month'])
data_expense_hol['Product_ID'] = pd.get_dummies(data_expense_hol['Product_ID'])

train_data = data_expense_hol[:388]
test_data = data_expense_hol[388:]

clf1 = LinearRegression()
clf1.fit(train_data.drop(['Sales'], axis = 1), train_data['Sales'])
clf1.score(train_data.drop(['Sales'], axis = 1), train_data['Sales'])
pred = clf1.predict(test_data.drop(['Sales'], axis = 1))
test['Sales'] = pred
test.to_csv('linear.csv')

clf2 = RF(n_estimators = 100, max_depth = 7)
clf2.fit(train_data.drop(['Sales', 'Year'], axis = 1), train_data['Sales'])
print(clf2.score(train_data.drop(['Sales', 'Year'], axis = 1), train_data['Sales']))
pred = clf2.predict(test_data.drop(['Sales', 'Year'], axis = 1))
test['Sales'] = pred
test.to_csv('random2.csv')
def SMAPE(x, y):
    return np.average(abs(x - y)/((abs(x) + abs(y))/2)) * 100


0.992410648736
